# Day 20 - Multivariate Stats

### Multiple Linear Regression

We have talked about linear regression in past modules. However, this concerns one Y and one X. Multiple Linear Regression (MLR) is a linear approach to modeling the relationship between a label and two-to-many features.

MLR produces coefficients (i.e., betas, β) for each of the features and a label (y)-intercept. This is much like the formula you learned in high school for a straight line (y = mx + b), except there are many x features and we refer to m (slope) as betas (β). For example, if you generated a model to predict someone’s income (y) based on their age (x1), education (x2), and years of work experience (x3), MLR would produce an expression of the following form:

y = β1x1 + β2x2 + β3x3 + b

The β coefficients are essentially weights that indicate how important the value of each feature is.

In [7]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

df = pd.read_csv('data/insurance.csv')

# Set label and features
y = df['charges']                    
X = df.select_dtypes(np.number).assign(const=1)
X = X.drop(columns=['charges'])
X.head()  # Show first 5 rows of X in order to see what we have created

,age,bmi,children,const
0,19,27.900,0,1
1,18,33.770,1,1
2,28,33.000,3,1
3,33,22.705,0,1
4,32,28.880,0,1


In [8]:
# Run the multiple linear regression model
model = sm.OLS(y, X)
results = model.fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.120
Model:                            OLS   Adj. R-squared:                  0.118
Method:                 Least Squares   F-statistic:                     60.69
Date:                Mon, 01 Apr 2024   Prob (F-statistic):           8.80e-37
Time:                        15:14:33   Log-Likelihood:                -14392.
No. Observations:                1338   AIC:                         2.879e+04
Df Residuals:                    1334   BIC:                         2.881e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
age          239.9945     22.289     10.767      0.0

#### Descriptives

The upper-left quadrant contains some basic descriptive information, including dependent variable (label), modeling formula used (ordinary least squares [OLS]), method of estimation (Least Squares), date, time, sample size (Number of Observations), degrees of freedom defined for the model (n = number of features (k)), and residuals/error (n = sample size − k − 1).

#### Overall Model Quality or "Fit"
In the upper-right quadrant, notice our new effect size measure: R-squared = 0.12. That means our model, based on the variables age, bmi, and children, can explain 12% of the variance in insurance charges. The first question I always get from students is, “Is that good?” The answer is, as usual, “it depends.” You can’t compare the R2 of this model to, say, a model predicting the number of insurance claims. The count of claims is a much simpler measure than actual charges and would likely be easier to predict and, therefore, have a higher R2 value. No, you can only evaluate an R2 value relative to other models predicting the same thing. Perhaps the best anyone has ever predicted insurance charges in the past was 10%. In that case, 12% is great!

Next, what is "Adj. R-squared?" Adjusted R-squared is a modified version of R-squared that has been adjusted for the number of predictors in the model. The adjusted R-squared increases only if the new term improves the model more than would be expected by chance. It decreases when a predictor improves the model by less than expected by chance. In the case above, these numbers are very close (0.118 approximately = 0.120). This means that the three features (age, bmi, and children) each account for a decent improvement in R2.

Hopefully you recognize the F-statistic from learning about one-way ANOVAs. As a reminder, the F-statistic is a value representing the ratio of differences among sample means divided by the combined variances within samples. Higher values conceptually represent the effect size of the independent variable(s) on the dependent variable. The Prob (F-statistic) simply refers to the p-value of the entire model. In the model above, the p-value is extremely small, indicating that the entire model is statistically significant. It is quite possible for the entire model to be statistically significant while none of the individual features have a significant p-value. This simply means that all features are required together to achieve a statistically significant model.

In [3]:
###looking for in-sample predictions
df_insample = pd.DataFrame({'Actual':df['charges'], 
                    'Predicted':results.fittedvalues, 
                    'Residuals (Error)':df['charges'] - results.fittedvalues})

df_insample.head(10)

,Actual,Predicted,Residuals (Error)
0,16884.92400,6908.777533,9976.146467
1,1725.55230,9160.977061,-7435.424761
2,4449.46200,12390.946918,-7941.484918
3,21984.47061,8543.527095,13440.943515
4,3866.85520,10354.147396,-6487.292196
5,3756.62160,9071.411158,-5314.789558
6,8240.58960,15771.234831,-7530.645231
7,7281.50560,12804.138689,-5522.633089
8,6406.41070,12955.328269,-6548.917569
9,28923.13692,16064.459249,12858.677671


In [4]:
df_insample['Absolute Residual'] = abs(df_insample['Residuals (Error)'])
df_insample['Squared Residual'] = df_insample['Residuals (Error)']**2

df_insample.head()

,Actual,Predicted,Residuals (Error),Absolute Residual,Squared Residual
0,16884.92400,6908.777533,9976.146467,9976.146467,9.952350e+07
1,1725.55230,9160.977061,-7435.424761,7435.424761,5.528554e+07
2,4449.46200,12390.946918,-7941.484918,7941.484918,6.306718e+07
3,21984.47061,8543.527095,13440.943515,13440.943515,1.806590e+08
4,3866.85520,10354.147396,-6487.292196,6487.292196,4.208496e+07


Now we are ready to calculate two very practical measures of model fit:

- Mean absolute error (MAE) is the average of the absolute value of the difference of all predicted y values from the actual y values. Therefore, lower numbers are better.

- Root mean squared error (RMSE) is conceptually similar to MAE. However, rather than using the absolute value of differences between the actual and predicted y values, those differences are squared and summed. RMSE is the square root of that sum. Again, lower numbers are better.

In other words, MAE and RMSE are two measures that answer the question of "how far off are out predictions on average?" We can calculate these very simply in Python using our df_residuals object:

In [48]:
print(f"MAE:\t{round(df_insample['Absolute Residual'].mean(), 4)}")
print(f"RMSE:\t{round(df_insample['Squared Residual'].mean() ** (1/2), 4)}")

MAE:	9015.4422
RMSE:	11355.3179


If MAE and RMSE are conceptually similar, which should you use? Because RMSE squares the errors, it will amplify the effect of relatively larger residuals. This is useful when the data are not normally distributed. Therefore, to the degree that the data are not normally distributed, RMSE will be increasingly larger than MAE. In most cases, there is some degree of abnormality. Therefore, we usually prefer RMSE to MAE just to be on the "safe" side.

#### Categorical Variables

Our MLR did not, and could not, use any of the categorical variables in our dataset. In order to use categorical variables, we have to turn them into dummy codes, which are binary (0/1) variables that equally represent each group in a categorical feature.

We can modify our prior code to generate dummy variables in several ways using Pandas. Let's begin with the simplest technique by manually choosing our features to dummy code:

In [49]:
# Manually enter column names to dummy code
df2 = pd.get_dummies(df, columns=['sex', 'smoker', 'region'], dtype=int)
df2.head()


,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,1,0,0,1,0,0,0,1
1,18,33.770,1,1725.55230,0,1,1,0,0,0,1,0
2,28,33.000,3,4449.46200,0,1,1,0,0,0,1,0
3,33,22.705,0,21984.47061,0,1,1,0,0,1,0,0
4,32,28.880,0,3866.85520,0,1,1,0,0,1,0,0


In [50]:
# Set label and features
y = df2['charges']
X = df2.drop(columns=['charges']).assign(const=1)

# Run the multiple linear regression model
model = sm.OLS(y, X).fit()
print(model.summary())  # View results

                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.751
Model:                            OLS   Adj. R-squared:                  0.749
Method:                 Least Squares   F-statistic:                     500.8
Date:                Wed, 06 Dec 2023   Prob (F-statistic):               0.00
Time:                        13:38:34   Log-Likelihood:                -13548.
No. Observations:                1338   AIC:                         2.711e+04
Df Residuals:                    1329   BIC:                         2.716e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
age                256.8564     11.899  

In [51]:
df3 = pd.get_dummies(df, columns=df.select_dtypes(['object']).columns, dtype=int, drop_first=True)
df3.head()

,age,bmi,children,charges,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,0,1,0,0,1
1,18,33.770,1,1725.55230,1,0,0,1,0
2,28,33.000,3,4449.46200,1,0,0,1,0
3,33,22.705,0,21984.47061,1,0,1,0,0
4,32,28.880,0,3866.85520,1,0,1,0,0


In [52]:
# Set label and features
y = df3['charges']
X = df3.drop(columns=['charges']).assign(const=1)

# Run the multiple linear regression model
model = sm.OLS(y, X).fit()
print(model.summary())  # View results

                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.751
Model:                            OLS   Adj. R-squared:                  0.749
Method:                 Least Squares   F-statistic:                     500.8
Date:                Wed, 06 Dec 2023   Prob (F-statistic):               0.00
Time:                        13:38:36   Log-Likelihood:                -13548.
No. Observations:                1338   AIC:                         2.711e+04
Df Residuals:                    1329   BIC:                         2.716e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
age                256.8564     11.899  

In [53]:
print(f"MAE:\t{round(abs(model.fittedvalues - y).mean(), 4)}")
print(f"RMSE:\t{round(((model.fittedvalues - y)**2).mean() ** (1/2), 4)}")


MAE:	4170.8869
RMSE:	6041.6797
